In [2]:
# 同stars指标情况相同
# 希望能够计算每个项目的participant总数
# 并筛选出participant总数排名前十的项目

import pymysql
import json

# 连接到 MySQL 数据库
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='cptbtptp789',
    database='project_data',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 删除原表 participants_total_metrics（如果存在）
drop_table_query_participants_total = "DROP TABLE IF EXISTS participants_total_metrics"
try:
    cursor.execute(drop_table_query_participants_total)
    print("原表 participants_total_metrics 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表 participants_total_metrics 时出错: {e}")

# 删除原表 top_10_participants_projects（如果存在）
drop_table_query_top_10 = "DROP TABLE IF EXISTS top_10_participants_projects"
try:
    cursor.execute(drop_table_query_top_10)
    print("原表 top_10_participants_projects 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表 top_10_participants_projects 时出错: {e}")

# 创建新表 participants_total_metrics
create_table_query_participants_total = """
CREATE TABLE IF NOT EXISTS participants_total_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    total_participants INT
)
"""
try:
    cursor.execute(create_table_query_participants_total)
    print("新表 participants_total_metrics 创建成功")
except pymysql.Error as e:
    print(f"创建新表 participants_total_metrics 时出错: {e}")

# 从 participants_metrics 表中查询数据
select_query_participants_total = "SELECT project_name, metric_value FROM participants_metrics"
cursor.execute(select_query_participants_total)

# 获取查询结果
results_participants_total = cursor.fetchall()

# 处理并插入数据到 participants_total_metrics 表
unique_projects = set()
for row in results_participants_total:
    project_name, metric_value_str = row
    if project_name not in unique_projects:
        metric_value = json.loads(metric_value_str)
        total_participants = 0
        for value in metric_value.values():
            if isinstance(value, (int, float)):
                total_participants += value
        insert_query_participants_total = "INSERT INTO participants_total_metrics (project_name, total_participants) VALUES (%s, %s)"
        try:
            cursor.execute(insert_query_participants_total, (project_name, total_participants))
            conn.commit()
            unique_projects.add(project_name)
        except pymysql.Error as e:
            print(f"插入数据到 participants_total_metrics 表时出错: {e}")

# 创建新表 top_10_participants_projects
create_table_query_top_10 = """
CREATE TABLE IF NOT EXISTS top_10_participants_projects (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    total_participants INT
)
"""
try:
    cursor.execute(create_table_query_top_10)
    print("新表 top_10_participants_projects 创建成功")
except pymysql.Error as e:
    print(f"创建新表 top_10_participants_projects 时出错: {e}")

# 从 participants_total_metrics 表中筛选出前 10 个参与者总数最高的项目
select_query_top_10 = """
SELECT project_name, total_participants
FROM participants_total_metrics
ORDER BY total_participants DESC
LIMIT 10
"""
cursor.execute(select_query_top_10)

# 获取查询结果
results_top_10 = cursor.fetchall()

# 处理并插入数据到 top_10_participants_projects 表
for row in results_top_10:
    project_name, total_participants = row
    insert_query_top_10 = "INSERT INTO top_10_participants_projects (project_name, total_participants) VALUES (%s, %s)"
    try:
        cursor.execute(insert_query_top_10, (project_name, total_participants))
        conn.commit()
    except pymysql.Error as e:
        print(f"插入数据到 top_10_participants_projects 表时出错: {e}")

# 关闭连接
cursor.close()
conn.close()

原表 participants_total_metrics 已删除（如果存在）
原表 top_10_participants_projects 已删除（如果存在）
新表 participants_total_metrics 创建成功
新表 top_10_participants_projects 创建成功
